# **Lab 2: Matrix factorization**
**Fabián Levicán**

# **Abstract**

This is the second lab in the course DD2363 Methods in Scientific Computing. It is about using Jupyter to implement three methods: one to calculate a sparse matrix-vector product, one to obtain the QR factorization of a matrix, and a direct solver of a linear system of equations. Some objectives may be to understand the benefits of sparse matrix representation, to become familiar with at least one QR factorization algorithm, and to understand how to use this algorithm to implement a direct solver. The results of these methods are then tested in various ways, and they are favorable.

#**About the code**

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Fabián Levicán (fils2@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import math
import time
import numpy as np

# **Introduction**

In scientific computing one often encounters matrices with very few non-zero entries. These matrices are much more efficiently manipulated with a sparse matrix representation, which here consists of three vectors, than with the usual dense matrix representation. Here we implement a method to calculate a matrix-vector product given a matrix in such a representation.

Any real square matrix $A$ can be factorized as $A = QR$, with $Q$ orthogonal, and $R$ upper triangular. There are various algorithms that may be used to obtain this factorization, including the Gram-Schmidt algorithm, Householder reflections, and Givens rotations. Each has its advantages and disadvantages. Here we implement the Gram-Schmidt algorithm, as the author was already familiar with it, and thus it was the simplest approach. 

To solve a linear system $Ax = b$ one may obtain the QR factorization of $A$ and use the property of orthogonal matrices that $Q^{-1} = Q^T$ to get $Rx = Q^Tb$, and then use backward substitution to solve for $x$. This is a direct solver, because intermediate steps don't approximate $x$, and is implemented here.

# **Methods**

A sparse matrix $A$ consists of three vectors:
* $A[0]$, or *val*, which stores the entries of $A$ starting from the top left and moving from left to right, and then from top to bottom.
* $A[1]$, or *col_idx*, where $A[1][i]$ stores the index of the column (in the dense matrix $A$) where $A[0][i]$ is supposed to go.
* $A[2] (\in \mathbb{R}^{m + 1})$, or *row_ptr*, where $A[2][i]$ stores the index $j$ where $A[0][j]$ is the leftmost entry in the $i$-th row of the dense matrix $A$. $A[2][i] = A[2][i - 1]$ if all the entries in the $i$-th row of the dense matrix $A$ are $0$, $A[2][0] = 0$ if all the entries in the $0$-th row of the dense matrix $A$ are $0$, and $A[2][m] = len(A[0]) = len(val)$.

The method sparseMatrixVectorProduct takes a sparse matrix $A$, a vector $x$, the number of rows ($m$) of the dense matrix $A$, and the number of columns ($n$) of the dense matrix $A$ as input, and returns the matrix-vector product $b = Ax$.

The algorithm iterates ($i$) through the number of rows of the dense matrix $A$, takes the relevant Python slice of *col_idx*, iterates ($j$) through the indices of this slice, and adds $val[row\_ptr[i] + j] \cdot x[slice[j]]$ to the relevant entry of $b$. It is easy to verify that this is correct by definition.

In [0]:
def sparseMatrixVectorProduct(A, x, m, n):
  val = A[0]
  col_idx = A[1]
  row_ptr = A[2]
  b = np.zeros(m)

  for i in range(m):
    col_idx_slice = col_idx[row_ptr[i]:row_ptr[i + 1]]
    for j in range(len(col_idx_slice)):
      b[i] += val[row_ptr[i] + j]*x[col_idx_slice[j]]  
  return b

The method qrFact takes a real square matrix $A \in \mathbb{R}^{n \times n}$ as input, and returns a tuple $(Q, R)$ containing its QR factorization.

The algorithm computes (in order), for every column $a_i$ of $A$, the vectors $Q[i] = a_i - \sum_{j = 0}^{i - 1} proj_{Q[j]}a_i$, where $proj_{u}v$ for $u, v \in \mathbb{R}^n$ is the projection of $v$ on $u$ defined as $\frac{\langle u, v \rangle}{\langle u, u \rangle}u$. The algorithm then normalizes these vectors and transposes the matrix $Q$. Finally, the upper triangular $R$ is obtained by noting that $R[i][j] = \langle e_i, a_j \rangle$, where $e_i$ is the $i$-th column of $Q$. This is the Gram-Schmidt algorithm. The author used [this](https://en.wikipedia.org/wiki/QR_decomposition) Wikipedia article as reference.

Finally, note that $A$ is singular, then if the $i$-th column of $A$ is linearly dependent with the previous ones, then $Q[i]$ is the zero vector, and here the algorithm would raise an error. To avoid this, the algorithm simply removes $Q[i]$ from the $Q$ matrix (this can be done because we want to obtain an orthonormal basis for the column space of $A$). Here we have two options:

* Output $Q \in \mathbb{R}^{n \times dim(CS(A))}$, where $CS(A)$ is the column space of $A$, and thus output $R \in \mathbb{R}^{dim(CS(A)) \times n}$.
* Output $Q \in \mathbb{R}^{n^2}$, but artificially choose the remaining $n - dim(CS(A))$ orthonormal vectors.

The choice is inconsequential (in either case $A = QR$), but author decided to go with the first option, as he thinks it is more important that $Q$ encodes an orthonormal basis of the column space of $A$ than that $Q$ and $R$ are square.

In [0]:
def qrFact(A):
  n = A.shape[0]
  A = np.transpose(A)
  Q = np.empty((n, n))

  # Calculate the Q matrix
  sum = np.empty(n)
  zeroQis = list()
  for i in range(n):
    sum[:] = 0
    for j in range(i):
      if(j not in zeroQis):
        sum += (Q[j] @ A[i])/(Q[j] @ Q[j])*Q[j]
    Q[i] = A[i] - sum

    # Take note of the indices i such that Q[i] is the zero vector
    if(np.isclose(Q[i], np.zeros(n)).all()):
      zeroQis.append(i)

  # Normalize the non-zero Q[i]s
  for i in range(n):
    if(i not in zeroQis):
      Q[i] = Q[i]/np.linalg.norm(Q[i])
  
  # Delete the zero Q[i]s
  finalQ = list()
  for i in range(n):
    if(not np.isclose(Q[i], np.zeros(n)).all()):
      finalQ.append(Q[i])
  finalQ = np.array(finalQ)

  # Calculate the R matrix
  R = np.zeros((n, len(finalQ)))
  for i in range(n):
    for j in range(i + 1):
      if(j < len(finalQ)):
        R[i][j] = finalQ[j] @ A[i]

  # Transpose both matrices
  finalQ = np.transpose(finalQ)
  R = np.transpose(R)

  return (finalQ, R)

The method directSolve takes a square matrix $A \in \mathbb{R}^{n \times n}$, and a vector $b \in \mathbb{R}^n$ as input, and returns a vector $x \in \mathbb{R}^n$ such that $Ax = b$.

To do this, we obtain the QR factorization of $A$ and use the property of orthogonal matrices that $Q^{-1} = Q^T$ to get $Rx = Q^Tb$, and then use the usual backward substitution to solve for $x$.

In [0]:
def directSolve(A, b):
  n = A.shape[0]
  qrFactA = qrFact(A)
  rhs = np.transpose(qrFactA[0]) @ b
  R = qrFactA[1]
  x = np.empty(n)

  x[n - 1] = rhs[n - 1]/R[n - 1][n - 1]
  for i in range(n - 2, -1, -1):
    sum = 0
    for j in range(i + 1, n):
      sum += R[i][j]*x[j]
    x[i] = (rhs[i] - sum)/R[i][i]

  return x

# **Results**

The method sparseMatrixVectorProduct is tested first with a matrix obtained from class, and then with two matrices obtained from [this](https://en.wikipedia.org/wiki/Sparse_matrix) Wikipedia article. Test $2$ also contains the second special case mentioned in the third point of the list in the description for the method.

The tests consist simply of comparing the results of the sparse matrix-vector product and the dense matrix-vector product entry-by-entry.

In [6]:
# sparseMatrixVectorProduct tests

# Test 1: Matrices A obtained from class

val = [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3]
col_idx = [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5]
row_ptr = [0, 3, 5, 6, 8, 9, 11]
ASparse = np.array([val, col_idx, row_ptr])
x = np.array([1, 1, 1, 1, 1, 1])
m = 6
n = 6

ADense = np.array([[3, 2, 0, 2, 0, 0],
                  [0, 2, 1, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0],
                  [0, 0, 3, 2, 0, 0],
                  [0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 2, 3]])

assert((sparseMatrixVectorProduct(ASparse, x, m, n) == ADense @ x).all())

# Test 2: Matrices A obtained from Wikipedia

val = [5, 8, 3, 6]
col_idx = [0, 1, 2, 1]
row_ptr = [0, 0, 2, 3, 4]
ASparse = np.array([val, col_idx, row_ptr])
x = np.array([2, 3, 5, 7])
m = 4
n = 4

ADense = np.array([[0, 0, 0, 0],
                   [5, 8, 0, 0],
                   [0, 0, 3, 0],
                   [0, 6, 0, 0]])

assert((sparseMatrixVectorProduct(ASparse, x, m, n) == ADense @ x).all())

# Test 3: Matrices A obtained from Wikipedia

val = [10, 20, 30, 40, 50, 60, 70, 80]
col_idx = [0, 1, 1, 3, 2, 3, 4, 5]
row_ptr = [0, 2, 4, 7, 8]
ASparse = np.array([val, col_idx, row_ptr])
x = np.array([-1, -0.5, 0, 0.5, 1, 1.5])

ADense = np.array([[10, 20, 0, 0, 0, 0],
                   [0, 30, 0, 40, 0, 0],
                   [0, 0, 50, 60, 70, 0],
                   [0, 0, 0, 0, 0, 80]])

assert((sparseMatrixVectorProduct(ASparse, x, m, n) == ADense @ x).all())

print("Tests passed successfully!")


Tests passed successfully!


The method qrFact is tested with four matrices $A$, two non-singular (see `# Test 1` and `# Test 4`), and two singular (see `# Test 2` and `# Test 3`). Each test consists of three assertions:

* The $R$ matrix returned by the method (`qrFactA[1]`) is the same as its upper triangular part. In the case of $A$ singular, NumPy considers only the first $dim(CS(A))$ rows, as $R \in \mathbb{R}^{dim(CS(A)) \times n}$.
* The Frobenius norm $\|Q^TQ - I_n\|_F = 0$ within a tolerance. In the case of $A$ singular, $I_n = I_{dim(CS(A))}$ instead (it can be shown that $QQ^T$ is not necessarily equal to $I_n$, i.e., that $Q^T$ is a left inverse of $Q$, but not a right inverse).
* The Frobenius norm $\|QR - A\|_F = 0$ within a tolerance.

In [7]:
# qrFact tests

# Test 1: Matrix A obtained from Wikipedia

A1 = np.array([[12, -51, 4],
               [6, 167, -68],
               [-4, 24, -41]])

qrFactA = qrFact(A1)

# Check if R is upper triangular

assert((qrFactA[1] == np.triu(qrFactA[1])).all())

# Frobenius norm tests

assert(np.linalg.norm(np.transpose(qrFactA[0]) @ qrFactA[0] - np.identity(A1.shape[0])) < 0.0001) #QTQ - I
assert(np.linalg.norm(qrFactA[0] @ qrFactA[1] - A1) < 0.0001) #QR - A

# Test 2: Singular matrix

A2 = np.array([[1, 2, 3, 4],
               [5, 6, 7, 8],
               [-1, -2, -3, -4],
               [0.2, 0.3, -0.2, -0.3]])

qrFactA = qrFact(A2)

# Check if R is upper triangular

assert((qrFactA[1] == np.triu(qrFactA[1])).all())

# Frobenius norm tests

assert(np.linalg.norm(np.transpose(qrFactA[0]) @ qrFactA[0] - np.identity(qrFactA[0].shape[1])) < 0.0001) #QTQ - I
assert(np.linalg.norm(qrFactA[0] @ qrFactA[1] - A2) < 0.0001) #QR - A

# Test 3: Singular matrix

A3 = np.array([[0, 1, 0, 1],
               [1, 0, 1, 0],
               [0, 1, 0, 1],
               [1, 0, 1, 0]])

qrFactA = qrFact(A3)

# Check if R is upper triangular

assert(np.allclose(qrFactA[1], np.triu(qrFactA[1])))

# Frobenius norm tests

assert(np.linalg.norm(np.transpose(qrFactA[0]) @ qrFactA[0] - np.identity(qrFactA[0].shape[1])) < 0.0001) #QTQ - I
assert(np.linalg.norm(qrFactA[0] @ qrFactA[1] - A3) < 0.0001) #QR - A

# Test 4: Non-singular matrix

A4 = np.array([[0, 1, 0, 1],
               [1, 0, 1, 0],
               [0, 1, 1, 1],
               [1, 1, 1, 0]])

qrFactA = qrFact(A4)

# Check if R is upper triangular

assert((qrFactA[1] == np.triu(qrFactA[1])).all())

# Frobenius norm tests

assert(np.linalg.norm(np.transpose(qrFactA[0]) @ qrFactA[0] - np.identity(A4.shape[0])) < 0.0001) #QTQ - I
assert(np.linalg.norm(qrFactA[0] @ qrFactA[1] - A4) < 0.0001) #QR - A

print("Tests passed successfully!")

Tests passed successfully!


The method directSolve is tested by solving the linear equations $Ax = b$ where the $A$s are the non-singular matrices defined in the tests for the previous method. The method is tested with two $b$ vectors for each $A$. Each test consists of two assertions:

* The residual $\|Ax - b\| = 0$ within a tolerance.
* The Euclidian norm $\|x - y\| = 0$ within a tolerance, where $y$ is NumPy's solution to the associated linear equation.

In [8]:
# directSolve tests

# Test 1

b1 = [1, 1, 1]

# norm(Ax - b) test

assert(np.linalg.norm(A1 @ directSolve(A1, b1) - b1) < 0.0001)

# norm(x - y) test

assert(np.linalg.norm(directSolve(A1, b1) - np.linalg.solve(A1, b1)) < 0.0001)

# Test 2

b2 = [-1, 0, 1]

# norm(Ax - b) test

assert(np.linalg.norm(A1 @ directSolve(A1, b2) - b2) < 0.0001)

# norm(x - y) test

assert(np.linalg.norm(directSolve(A1, b2) - np.linalg.solve(A1, b2)) < 0.0001)

# Test 3

b3 = [0.2, 0.3, -0.2, -0.3]

# norm(Ax - b) test

assert(np.linalg.norm(A4 @ directSolve(A4, b3) - b3) < 0.0001)

# norm(x - y) test

assert(np.linalg.norm(directSolve(A4, b3) - np.linalg.solve(A4, b3)) < 0.0001)

# Test 4

b4 = [-1, 0, 1, 2]

# norm(Ax - b) test

assert(np.linalg.norm(A4 @ directSolve(A4, b4) - b4) < 0.0001)

# norm(x - y) test

assert(np.linalg.norm(directSolve(A4, b4) - np.linalg.solve(A4, b4)) < 0.0001)

print("Tests passed successfully!")

Tests passed successfully!


# **Discussion**

All the results were favorable.

The author spent most of his time dealing with the singular case in the Gram-Schmidt algorithm for QR factorization, and revisited many elementary facts from linear algebra. His conclusion about this is that the theoretical aspects of these algorithms, even if sometimes ill-motivated or arbitrary from an applied perspective, can have important practical implications, especially while dealing with edge cases.

The author also appreciates how easy it is to turn a QR factorization algorithm into a direct solver for linear equations.